In [ ]:
#r "nuget:Microsoft.Spark"
using System;
using System.Collections.Generic;
using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;
using static Microsoft.Spark.Sql.Functions;


Environment.SetEnvironmentVariable("EVENT_HUB_CONNECTION_STRING", "Endpoint=sb://evhns-timeseries-timeseries-u-001.servicebus.windows.net/;SharedAccessKeyName=listen;SharedAccessKey=lGW5ENgWGFeRhNNBhPkq4uAYrebkWhSTa0ZbvafB9cw=;EntityPath=received-timeseries");
/////////////////////////////////////////////////////

var spark = SparkSession.Builder().GetOrCreate();
var timeseries_unprocessed_path = "TODO";

var eventHubConnectionString = Environment.GetEnvironmentVariable("EVENT_HUB_CONNECTION_STRING");

// TODO BJARKE: Extract hostname from connection string: Endpoint=sb://evhns-integrationstest-u-002.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=*******
string bootstrapServers = "evhns-timeseries-timeseries-u-001.servicebus.windows.net:9093"; // 9093 is the port used to communicate with Event Hubs, see [troubleshooting guide](https://docs.microsoft.com/azure/event-hubs/troubleshooting-guide)
string eh_sasl = $"org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{eventHubConnectionString}\";";
Console.WriteLine(eh_sasl);

var streamingDf = spark
    .ReadStream()
    .Format("kafka")
    .Option("kafka.bootstrap.servers", bootstrapServers)
    .Option("subscribe", "received-timeseries")
    .Option("kafka.sasl.mechanism", "PLAIN")
    .Option("kafka.security.protocol", "SASL_SSL")
    .Option("kafka.sasl.jaas.config", eh_sasl)
    .Option("kafka.request.timeout.ms", "60000")
    .Option("kafka.session.timeout.ms", "60000")
    .Option("failOnDataLoss", "false")
    .Option("checkpointLocation", "/tmp/kafka_cp.txt")
    .Load()
    .SelectExpr("CAST(value AS STRING)");

    var checkpointPath = "/tmp/received_time_series_checkpoint";
    //var checkpointPath = $"abfss://{delta_lake_container_name}@{storage_account_name}.dfs.core.windows.net/checkpoint"
     streamingDf
        .WriteStream()
        .Option("checkpointLocation", checkpointPath)
        .ForeachBatch((df, epochId) => ProcessEventhubItem(df, epochId, timeseries_unprocessed_path))
        .Start().AwaitTermination();
    spark.Stop();

  //  var dfWrite = streamingDf.WriteStream().OutputMode("append").Format("console").Start();
    //dfWrite.AwaitTermination();
    //spark.Stop();
   
    public static void ProcessEventhubItem(DataFrame df, long epochId, string timeseriesUnprocessedPath)
    {
        Console.WriteLine("Hello");
        if(df.Head(1).Count()>0){
            df.Show(20,100);
        }
        // if (df == null) throw new ArgumentNullException(nameof(timeseriesUnprocessedPath));
        // df = df
        //     .WithColumn("year", Functions.Year(df["enqueuedTime"]))
        //     .WithColumn("month", Functions.Month(df["enqueuedTime"]))
        //     .WithColumn("day", Functions.DayOfMonth(df["enqueuedTime"]));
        // // df
        // //     .Write()
        // //     .PartitionBy("year", "month", "day")
        // //     .Format("delta")
        // //     .Mode(SaveMode.Append)
        // //     .Save(timeseries_unprocessed_path);
        // df.Show();
    }

Installed Packages Microsoft.Spark, 2.1.0

Error: (53,25): error CS1503: Argument 2: cannot convert from 'bool' to 'int'